In The Name Of GOD

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving IOT-temp.csv to IOT-temp.csv


In [ ]:
import io
data = pd.read_csv(io.BytesIO(uploaded['IOT-temp.csv']))

In [ ]:
#random 5 rows
data.sample(5)

,id,room_id/id,noted_date,temp,out/in
38275,__export__.temp_log_61983_87453d0f,Room Admin,17-10-2018 15:54,39,Out
84931,__export__.temp_log_3156_fe5bf907,Room Admin,09-09-2018 15:30,28,Out
6996,__export__.temp_log_173134_f25b7af2,Room Admin,01-12-2018 20:18,36,Out
64500,__export__.temp_log_13167_b2f3faf7,Room Admin,11-09-2018 22:56,28,Out
94562,__export__.temp_log_130297_7de11613,Room Admin,03-09-2018 12:42,31,Out


In [ ]:
data.head()

,id,room_id/id,noted_date,temp,out/in
0,__export__.temp_log_196134_bd201015,Room Admin,08-12-2018 09:30,29,In
1,__export__.temp_log_196131_7bca51bc,Room Admin,08-12-2018 09:30,29,In
2,__export__.temp_log_196127_522915e3,Room Admin,08-12-2018 09:29,41,Out
3,__export__.temp_log_196128_be0919cf,Room Admin,08-12-2018 09:29,41,Out
4,__export__.temp_log_196126_d30b72fb,Room Admin,08-12-2018 09:29,31,In


In [ ]:
#Data set size
print(f'The data set have {data.shape[0]} rows and {data.shape[1]} columns')

The data set have 97606 rows and 5 columns


In [ ]:
data.describe()

,temp
count,97606.000000
mean,35.053931
std,5.699825
min,21.000000
25%,30.000000
50%,35.000000
75%,40.000000
max,51.000000


In [ ]:
#unique values in each columns
for i in data.columns:
  print(i + ":" , len(set(data[i])))

id: 97605
room_id/id: 1
noted_date: 27920
temp: 31
out/in: 2


In [ ]:
data.dtypes

id            object
room_id/id    object
noted_date    object
temp           int64
out/in        object
dtype: object

In [ ]:
#count missing values of each columns
data.isnull().sum()

id            0
room_id/id    0
noted_date    0
temp          0
out/in        0
dtype: int64

In [ ]:
#drop "id" and "room_id/id" columns
data_1 = data.drop(['id','room_id/id'],axis=1)
data_1.head()

,noted_date,temp,out/in
0,08-12-2018 09:30,29,In
1,08-12-2018 09:30,29,In
2,08-12-2018 09:29,41,Out
3,08-12-2018 09:29,41,Out
4,08-12-2018 09:29,31,In


In [ ]:
#separate date and time in noted_date column

date_list = []
time_list = []
for index, row in data_1.iterrows():
  date = (data_1["noted_date"][index]).split()[0]
  time = (data_1["noted_date"][index]).split()[1]
  date_list.append(date)
  time_list.append(time)

data_1['date'] = date_list
data_1['time'] = time_list

data_2 = data_1.drop('noted_date',axis=1)
data_2.head()

,temp,out/in,date,time
0,29,In,08-12-2018,09:30
1,29,In,08-12-2018,09:30
2,41,Out,08-12-2018,09:29
3,41,Out,08-12-2018,09:29
4,31,In,08-12-2018,09:29


In [ ]:
for i in data_1.columns:
  print(i + ":" , len(set(data_1[i])))

noted_date: 27920
temp: 31
out/in: 2
date: 86
time: 1440


#Feature extraction

In [ ]:
#separate year , month and day in date column
data_2['date'] = pd.to_datetime(data_2['date'])
data_2['year'] = data_2['date'].dt.year
data_2['month'] = data_2['date'].dt.month
data_2['day'] = data_2['date'].dt.day
data_3 = data_2.drop('date',axis=1)
data_3.head()

<ipython-input-14-21b3e0c7c163>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data_2['date'] = pd.to_datetime(data_2['date'])


,temp,out/in,time,year,month,day
0,29,In,09:30,2018,8,12
1,29,In,09:30,2018,8,12
2,41,Out,09:29,2018,8,12
3,41,Out,09:29,2018,8,12
4,31,In,09:29,2018,8,12


In [ ]:
for i in data_3.columns:
  print(i + ":" , len(set(data_3[i])))

temp: 31
out/in: 2
time: 1440
year: 1
month: 12
day: 24


In [ ]:
data_3 = data_3.drop('year',axis=1)
data_3.head()

,temp,out/in,time,month,day
0,29,In,09:30,8,12
1,29,In,09:30,8,12
2,41,Out,09:29,8,12
3,41,Out,09:29,8,12
4,31,In,09:29,8,12


#Data Cleaning

In [ ]:
#preprpcess for regression

def convert_time_to_float(df):
#feature extraction And feature reduction
  hour = []
  min = []
  for index, row in df.iterrows():
    hour.append(df["time"][index].split(":")[0])
    min.append(df["time"][index].split(":")[1])

  df["hour"] = hour
  df["min"] = min

  #convert strings to floats
  df['hour'] = df['hour'].astype(float)
  df['min'] = df['min'].astype(float)

  df_1 = df.drop('time' , axis =1)

  return df_1


#convert categorical column to numeric

def convert_out_in(df):
    df['out/in'].replace(['Out', 'In'],
                        [-1, 1], inplace=True)
    return df


In [ ]:
# Split data into train and test

from sklearn.model_selection import train_test_split

# ‘:’ stands for all
x = data_3.iloc[:,[1 ,2, 3, 4]]
y = data_3.iloc[:,0]


#train_test_split randomly distributes your data into training and testing set according to the ratio provided.
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [ ]:
x_train.head()

,out/in,time,month,day
12282,Out,09:42,11,21
80820,In,10:34,10,9
63439,In,00:33,12,9
17774,In,07:13,10,27
90088,In,10:02,6,9


In [ ]:
y_train.head()

12282    45
80820    31
63439    28
17774    32
90088    27
Name: temp, dtype: int64

In [ ]:
#outlier detection for 'month' column using IQR

import numpy as np

Q1 = x_train['month'].quantile(0.25)
Q3 = x_train['month'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR
#print(IQR)

#upper_array = np.where(x_train['month']>=upper)[0]
#lower_array = np.where(x_train['month']<=lower)[0]


x_train_1 = x_train[x_train.month < upper]
y_train_1 = y_train[x_train.month < upper]

y_train_1 = y_train_1[x_train_1.month > lower]
x_train_1 = x_train_1[x_train_1.month > lower]


x_test_1 = x_test
y_test_1 = y_test

print("old Shape: ", x_train.shape)
print("New Shape: ", x_train_1.shape)

x_train_1 = convert_time_to_float(x_train_1)
x_train_1.head()

old Shape:  (68324, 4)
New Shape:  (52221, 4)


,out/in,month,day,hour,min
12282,Out,11,21,9.0,42.0
80820,In,10,9,10.0,34.0
17774,In,10,27,7.0,13.0
20525,Out,10,21,1.0,23.0
39847,Out,10,17,14.0,44.0


In [ ]:
# outlier detection using Isolation forest
from sklearn.ensemble import IsolationForest

#convert strings to floats
x_train  = convert_time_to_float(x_train)
x_train['month'] = x_train['month'].astype(float)
x_train['day'] = x_train['day'].astype(float)
y_train = y_train.astype(float)



#select float columns of dataframe
x_and_y_train = x_train
x_and_y_train["temp"] = y_train
x_and_y_train_subset = x_and_y_train[["temp" , "month", "day", "hour", "min"]]


# Train the model
isf = IsolationForest(contamination=0.04)
isf.fit(x_and_y_train_subset)

# Predictions
# +1 means inlier
# -1 means outlier
predictions = isf.predict(x_and_y_train_subset)

# Extract predictions
x_and_y_train_subset["iso_forest_outliers"] = predictions

#replace “-1” with “Yes” and “1” with “No”
x_and_y_train_subset['iso_forest_outliers'] = x_and_y_train_subset['iso_forest_outliers'].replace([-1, 1], ["Yes", "No"])



#remove outliers
for index, row in x_and_y_train_subset.iterrows():
  if x_and_y_train_subset['iso_forest_outliers'][index] == "Yes" :
    x_train_2 = x_train.drop(index, axis=0)
    y_train_2 = y_train.drop(index, axis=0)

print("old Shape: ", x_train.shape)
print("New Shape: ", x_train_2.shape)

###############
x_test_2 = x_test
y_test_2 = y_test
###############


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


old Shape:  (68324, 6)
New Shape:  (68323, 6)


In [ ]:
x_train_2 = x_train_2.drop("temp",axis = 1)
x_train_2.head()

,out/in,month,day,hour,min
12282,Out,11.0,21.0,9.0,42.0
80820,In,10.0,9.0,10.0,34.0
63439,In,12.0,9.0,0.0,33.0
17774,In,10.0,27.0,7.0,13.0
90088,In,6.0,9.0,10.0,2.0


In [ ]:
x_test_2 = convert_time_to_float(x_test_2)

x_train_2 = convert_out_in(x_train_2)
x_test_2 = convert_out_in(x_test_2)

In [ ]:
x_test_1 = convert_time_to_float(x_test_1)

x_train_1 = convert_out_in(x_train_1)
x_test_1 = convert_out_in(x_test_1)

In [ ]:
# Linear Regressions

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train_1, y_train_1)
y_pred_1 = reg.predict(x_test_1)

reg = LinearRegression().fit(x_train_2, y_train_2)
y_pred_2 = reg.predict(x_test_2)

In [ ]:
#MSE

from sklearn.metrics import mean_squared_error
print("MSE of regression from IQR outlier detection :", mean_squared_error(y_test_1, y_pred_1) )
print("MSE of regression from isolation forest outlier detection :", mean_squared_error(y_test_2, y_pred_2) )

MSE of regression from IQR outlier detection : 16.51818018478585
MSE of regression from isolation forest outlier detection : 16.204101496111477


#Feature Selection and Feature Reduction


In [ ]:
x_test = convert_time_to_float(x_test)

In [ ]:
#x_train = convert_time_to_float(x_train)

x_train = convert_out_in(x_train)
x_test = convert_out_in(x_test)
x_train = x_train.drop("temp",axis = 1)


In [ ]:
x_train.head()

,out/in,month,day,hour,min
12282,-1,11.0,21.0,9.0,42.0
80820,1,10.0,9.0,10.0,34.0
63439,1,12.0,9.0,0.0,33.0
17774,1,10.0,27.0,7.0,13.0
90088,1,6.0,9.0,10.0,2.0


In [ ]:
#VarianceThreshold
x_train_3 = x_train

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(x_train_3)
x_train_3.head()

,out/in,month,day,hour,min
12282,-1,11.0,21.0,9.0,42.0
80820,1,10.0,9.0,10.0,34.0
63439,1,12.0,9.0,0.0,33.0
17774,1,10.0,27.0,7.0,13.0
90088,1,6.0,9.0,10.0,2.0


In [ ]:
#Tree-based feature selection

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(x_train, y_train)
print(clf.feature_importances_)
model = SelectFromModel(clf, prefit=True)
x_train_4 = model.transform(x_train)
x_test_4 = model.transform(x_test)
x_train_4.shape

[0.03084801 0.13113795 0.21619331 0.34747386 0.27434687]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


(68324, 3)

In [ ]:
# Linear Regressions

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x_train, y_train)
y_pred_3 = reg.predict(x_test)

reg = LinearRegression().fit(x_train_4, y_train)
y_pred_4 = reg.predict(x_test_4)

In [ ]:
#MSE

from sklearn.metrics import mean_squared_error
print("MSE of regression from method1 :", mean_squared_error(y_test, y_pred_3) )
print("MSE of regression from tree_based method :", mean_squared_error(y_test, y_pred_4) )

MSE of regression from method1 : 16.204106411806816
MSE of regression from tree_based method : 19.8362566192626


#feature extraction results

In [ ]:
from sklearn.model_selection import train_test_split

# ‘:’ stands for all
xx = data_3.iloc[:,[1 ,2, 3, 4]]
yy = data_3.iloc[:,0]


#train_test_split randomly distributes your data into training and testing set according to the ratio provided.
xx_train,xx_test,yy_train,yy_test=train_test_split(xx,yy,test_size=0.3)

In [ ]:
xx_train.head()

,out/in,time,month,day
46751,Out,06:51,10,17
2253,Out,17:40,6,12
53879,Out,22:51,9,27
86145,Out,14:19,9,9
308,Out,04:16,8,12


In [ ]:
xx_train = xx_train.drop(["out/in", "time"],axis=1)
xx_test = xx_test.drop(["out/in", "time"],axis=1)

In [ ]:
#xxx_train = convert_time_to_float(x_train)
#xxx_test = convert_time_to_float(x_test)

xxx_train = x_train
xxx_test = x_test
xxx_train = xxx_train.drop("out/in",axis=1)
xxx_test = xxx_test.drop("out/in",axis=1)

In [ ]:
xx_train.head()

,month,day
46751,10,17
2253,6,12
53879,9,27
86145,9,9
308,8,12


In [ ]:
# Linear Regressions

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(xx_train, y_train)
y_pred_5 = reg.predict(xx_test)

reg = LinearRegression().fit(xxx_train, y_train)
y_pred_6 = reg.predict(xxx_test)

In [ ]:
#MSE

from sklearn.metrics import mean_squared_error
print("MSE of regression from method1 :", mean_squared_error(y_test, y_pred_5) )
print("MSE of regression from extraction features hour and min  :", mean_squared_error(y_test, y_pred_6) )

MSE of regression from method1 : 31.88777977795209
MSE of regression from extraction features hour and min  : 19.592925793723442
